# Automação da Compra de VR/VA

## Objetivo
Automatizar o processo mensal de compra de VR (Vale Refeição), garantindo que cada colaborador receba o valor correto, considerando ausências, férias e datas de admissão ou desligamento e calendário de feriados.

## Descrição do Processo
- Consolidação de 5 bases separadas em uma única base final
- Tratamento de exclusões (diretores, estagiários, aprendizes, afastados, exterior)
- Validação e correção de datas inconsistentes
- Cálculo automatizado do benefício por colaborador
- Geração de planilha final para envio à operadora

## 1. Importação de Bibliotecas e Configurações Iniciais

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurações do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

## 2. Verificação de Arquivos Disponíveis

In [3]:
# Pega o diretório onde está rodando o notebook/script
BASE_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path().resolve()

# Define pastas relativas
DADOS_DIR = BASE_DIR / "Dados"
UPLOADS_DIR = BASE_DIR / "Uploads"

# Verificar arquivos disponíveis em Dados
print("Arquivos disponíveis na pasta Dados:")
if DADOS_DIR.exists():
    arquivos_dados = list(DADOS_DIR.glob("*.xlsx"))
    for arquivo in sorted(arquivos_dados):
        tamanho = arquivo.stat().st_size / 1024  # KB
        modificado = datetime.fromtimestamp(arquivo.stat().st_mtime)
        print(f"{arquivo.name}")
else:
    print("Pasta Dados não encontrada")

# Verificar arquivos disponíveis em Uploads
print("\nArquivos na pasta Uploads:")
if UPLOADS_DIR.exists():
    arquivos_uploads = list(UPLOADS_DIR.glob("*.xlsx"))
    for arquivo in sorted(arquivos_uploads):
        tamanho = arquivo.stat().st_size / 1024  # KB
        modificado = datetime.fromtimestamp(arquivo.stat().st_mtime)
        print(f"{arquivo.name}")
else:
    print("Pasta Uploads não encontrada")

Arquivos disponíveis na pasta Dados:
ADMISSOABRIL.xlsx
AFASTAMENTOS.xlsx
APRENDIZ.xlsx
ATIVOS.xlsx
Basediasuteis.xlsx
Basesindicatoxvalor.xlsx
DESLIGADOS.xlsx
ESTAGIO.xlsx
EXTERIOR.xlsx
FERIAS.xlsx
VRMENSAL05_2025.xlsx

Arquivos na pasta Uploads:
Pasta Uploads não encontrada


## 3. Carregamento das Bases de Dados

In [4]:
def carregar_bases():
    bases = {}
    
    arquivos_esperados = {
        'ativos': 'ATIVOS.xlsx',
        'ferias': 'FERIAS.xlsx',
        'desligados': 'DESLIGADOS.xlsx',
        'admissoes': 'ADMISSOABRIL.xlsx',
        'sindicato_valor': 'Basesindicatoxvalor.xlsx',
        'dias_uteis': 'Basediasuteis.xlsx',
        'afastamentos': 'AFASTAMENTOS.xlsx',
        'estagiarios': 'ESTAGIO.xlsx',
        'aprendizes': 'APRENDIZ.xlsx',
        'exterior': 'EXTERIOR.xlsx',
        'vr_mensal': 'VRMENSAL05_2025.xlsx'
    }
    
    
    for nome, arquivo in arquivos_esperados.items():
        try:
            # Tenta carregar do diretório atual primeiro
            if Path(arquivo).exists():
                df = pd.read_excel(arquivo)
                bases[nome] = df
                print(f" -  {arquivo}: {len(df)} registros")
            elif DADOS_DIR.exists() and (DADOS_DIR / arquivo).exists():
                df = pd.read_excel(DADOS_DIR / arquivo)
                bases[nome] = df
                print(f" -  {arquivo}: {len(df)} registros")
            elif UPLOADS_DIR.exists() and (UPLOADS_DIR / arquivo).exists():
                df = pd.read_excel(UPLOADS_DIR / arquivo)
                bases[nome] = df
                print(f" -  {arquivo}: {len(df)} registros")
            else:
                print(f"{arquivo}: Arquivo não encontrado")
                bases[nome] = pd.DataFrame()
        except Exception as e:
            print(f"{arquivo}: Erro ao carregar - {str(e)}")
            bases[nome] = pd.DataFrame()
    
    return bases

# Carregar todas as bases
bases = carregar_bases()

 -  ATIVOS.xlsx: 1815 registros
 -  FERIAS.xlsx: 80 registros
 -  DESLIGADOS.xlsx: 51 registros
 -  ADMISSOABRIL.xlsx: 83 registros
 -  Basesindicatoxvalor.xlsx: 5 registros
 -  Basediasuteis.xlsx: 5 registros
 -  AFASTAMENTOS.xlsx: 20 registros
 -  ESTAGIO.xlsx: 27 registros
 -  APRENDIZ.xlsx: 33 registros
 -  EXTERIOR.xlsx: 4 registros
 -  VRMENSAL05_2025.xlsx: 1860 registros


## 4. Análise Exploratória das Bases

In [5]:
def analisar_bases(bases):
    """Análise exploratória das bases carregadas"""
    print("📈 Análise das Bases Carregadas\n")
    
    for nome, df in bases.items():
        if not df.empty:
            print(f"📋 {nome.upper()}:")
            print(f"   Registros: {len(df)}")
            print(f"   Colunas: {list(df.columns)}")
            
            # Mostrar primeiras linhas
            if len(df) > 0:
                print(f"   Primeiras linhas:")
                print(df.head(2).to_string(index=False))
            print("\n" + "="*80 + "\n")
        else:
            print(f"{nome.upper()}: Base vazia ou não carregada\n")

# Executar análise
analisar_bases(bases)

📈 Análise das Bases Carregadas

📋 ATIVOS:
   Registros: 1815
   Colunas: ['MATRICULA', 'EMPRESA', 'TITULO DO CARGO', 'DESC. SITUACAO', 'Sindicato']
   Primeiras linhas:
 MATRICULA  EMPRESA            TITULO DO CARGO DESC. SITUACAO                                                                    Sindicato
     34941     1410          TECH RECRUITER II    Trabalhando SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMPRESAS PROC DADOS ESTADO DE SP.
     24401     1410 COORDENADOR ADMINISTRATIVO    Trabalhando         SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE DADOS RIO GRANDE DO SUL


📋 FERIAS:
   Registros: 80
   Colunas: ['MATRICULA', 'DESC. SITUACAO', 'DIAS DE FÉRIAS']
   Primeiras linhas:
 MATRICULA DESC. SITUACAO  DIAS DE FÉRIAS
     32104         Férias              10
     32761         Férias              10


📋 DESLIGADOS:
   Registros: 51
   Colunas: ['MATRICULA ', 'DATA DEMISSÃO', 'COMUNICADO DE DESLIGAMENTO']
   Primeiras linhas:
 MATRICULA  DATA DEMISSÃO COMUNICADO DE DESLIGAM

## 5. Limpeza e Padronização dos Dados

In [6]:
def limpar_e_padronizar_dados(bases):
    """Limpa e padroniza os dados das bases"""
    print("Limpeza e Padronização dos Dados\n")
    
    # Padronizar nomes de colunas para MATRICULA
    colunas_matricula = ['MATRICULA', 'Matricula', 'matricula', 'Cadastro']
    
    for nome, df in bases.items():
        if not df.empty:
            # Encontrar coluna de matrícula
            col_matricula = None
            for col in colunas_matricula:
                if col in df.columns:
                    col_matricula = col
                    break
            
            if col_matricula and col_matricula != 'MATRICULA':
                df.rename(columns={col_matricula: 'MATRICULA'}, inplace=True)
                print(f"{nome}: Coluna {col_matricula} renomeada para MATRICULA")
            
            # Remover linhas com MATRICULA vazia
            if 'MATRICULA' in df.columns:
                antes = len(df)
                df.dropna(subset=['MATRICULA'], inplace=True)
                df = df[df['MATRICULA'] != '']
                depois = len(df)
                if antes != depois:
                    print(f"{nome}: Removidas {antes - depois} linhas com MATRICULA vazia")
                
                # Converter MATRICULA para int quando possível
                try:
                    df['MATRICULA'] = pd.to_numeric(df['MATRICULA'], errors='coerce').astype('Int64')
                except:
                    pass
            
            # Atualizar a base
            bases[nome] = df
    
    return bases

# Executar limpeza
bases = limpar_e_padronizar_dados(bases)

Limpeza e Padronização dos Dados

exterior: Coluna Cadastro renomeada para MATRICULA


## 6. Identificação de Exclusões

In [7]:
def identificar_exclusoes(bases):
    """Identifica colaboradores que devem ser excluídos do benefício"""
    print("Identificando Exclusões\n")
    
    exclusoes = set()
    motivos_exclusao = {}
    
    # 1. Estagiários
    if not bases['estagiarios'].empty and 'MATRICULA' in bases['estagiarios'].columns:
        estagiarios = set(bases['estagiarios']['MATRICULA'].dropna())
        exclusoes.update(estagiarios)
        for mat in estagiarios:
            motivos_exclusao[mat] = 'Estagiário'
        print(f"Estagiários: {len(estagiarios)} exclusões")
    
    # 2. Aprendizes
    if not bases['aprendizes'].empty and 'MATRICULA' in bases['aprendizes'].columns:
        aprendizes = set(bases['aprendizes']['MATRICULA'].dropna())
        exclusoes.update(aprendizes)
        for mat in aprendizes:
            motivos_exclusao[mat] = 'Aprendiz'
        print(f"Aprendizes: {len(aprendizes)} exclusões")
    
    # 3. Afastamentos
    if not bases['afastamentos'].empty and 'MATRICULA' in bases['afastamentos'].columns:
        afastados = set(bases['afastamentos']['MATRICULA'].dropna())
        exclusoes.update(afastados)
        for mat in afastados:
            motivos_exclusao[mat] = 'Afastado'
        print(f"Afastados: {len(afastados)} exclusões")
    
    # 4. Exterior
    if not bases['exterior'].empty and 'Cadastro' in bases['exterior'].columns:
        exterior = set(bases['exterior']['Cadastro'].dropna())
        exclusoes.update(exterior)
        for mat in exterior:
            motivos_exclusao[mat] = 'Exterior'
        print(f"Exterior: {len(exterior)} exclusões")
    
    # 5. Diretores (identificar por cargo)
    if not bases['ativos'].empty and 'TITULO DO CARGO' in bases['ativos'].columns:
        diretores_mask = bases['ativos']['TITULO DO CARGO'].str.contains('DIRETOR', case=False, na=False)
        diretores = set(bases['ativos'][diretores_mask]['MATRICULA'].dropna())
        exclusoes.update(diretores)
        for mat in diretores:
            motivos_exclusao[mat] = 'Diretor'
        print(f"Diretores: {len(diretores)} exclusões")
    
    print(f"\nTotal de exclusões: {len(exclusoes)}")
    
    return exclusoes, motivos_exclusao

# Identificar exclusões
exclusoes, motivos_exclusao = identificar_exclusoes(bases)

Identificando Exclusões

Estagiários: 27 exclusões
Aprendizes: 33 exclusões
Afastados: 20 exclusões
Diretores: 0 exclusões

Total de exclusões: 80


## 7. Consolidação da Base Principal

In [8]:
def consolidar_base_principal(bases, exclusoes):
    """Consolida todas as bases em uma única base principal"""
    
    if bases['ativos'].empty:
        print("Base de ativos não disponível")
        return pd.DataFrame()
    
    base_principal = bases['ativos'].copy()
    
    # Remover exclusões
    if 'MATRICULA' in base_principal.columns:
        antes = len(base_principal)
        base_principal = base_principal[~base_principal['MATRICULA'].isin(exclusoes)]
        depois = len(base_principal)
    
    # Adicionar informações de férias
    if not bases['ferias'].empty and 'MATRICULA' in bases['ferias'].columns:
        base_principal = base_principal.merge(
            bases['ferias'][['MATRICULA', 'DIAS DE FÉRIAS']],
            on='MATRICULA',
            how='left'
        )
    
    # Adicionar informações de desligamento
    if not bases['desligados'].empty and 'MATRICULA' in bases['desligados'].columns:
        base_principal = base_principal.merge(
            bases['desligados'][['MATRICULA', 'DATA DEMISSÃO', 'COMUNICADO DE DESLIGAMENTO']],
            on='MATRICULA',
            how='left'
        )
    
    return base_principal

# Consolidar base principal
base_principal = consolidar_base_principal(bases, exclusoes)
print(f"\n📋 Base principal consolidada: {len(base_principal)} colaboradores elegíveis")


📋 Base principal consolidada: 1795 colaboradores elegíveis


## 8. Mapeamento de Sindicatos e Valores

In [9]:
def mapear_sindicatos_valores(bases):
    """Cria mapeamento de sindicatos para valores e dias úteis"""
    print("🗺️ Mapeando Sindicatos e Valores\n")
    
    # Mapeamento de valores por estado/sindicato
    valores_sindicato = {}
    if not bases['sindicato_valor'].empty:
        for _, row in bases['sindicato_valor'].iterrows():
            if pd.notna(row.get('ESTADO')) and pd.notna(row.get('VALOR')):
                valores_sindicato[row['ESTADO']] = float(row['VALOR'])
        print(f"💰 Valores por estado: {valores_sindicato}")
    
    # Mapeamento de dias úteis por sindicato
    dias_uteis_sindicato = {}
    if not bases['dias_uteis'].empty:
        # Assumindo que a primeira linha contém os cabeçalhos
        df_dias = bases['dias_uteis']
        if len(df_dias) > 1:
            for _, row in df_dias.iloc[1:].iterrows():
                sindicato = row.iloc[0] if pd.notna(row.iloc[0]) else None
                dias = row.iloc[1] if pd.notna(row.iloc[1]) else None
                if sindicato and dias:
                    try:
                        dias_uteis_sindicato[sindicato] = int(dias)
                    except:
                        pass
        print(f"📅 Dias úteis por sindicato: {len(dias_uteis_sindicato)} mapeamentos")
    
    return valores_sindicato, dias_uteis_sindicato

# Mapear sindicatos e valores
valores_sindicato, dias_uteis_sindicato = mapear_sindicatos_valores(bases)

🗺️ Mapeando Sindicatos e Valores

💰 Valores por estado: {}
📅 Dias úteis por sindicato: 4 mapeamentos


## 9. Cálculo de Dias Úteis por Colaborador

In [10]:
def calcular_dias_uteis_colaborador(base_principal, dias_uteis_sindicato):
    """Calcula os dias úteis para cada colaborador"""
    
    base_calculo = base_principal.copy()
    
    # Inicializar coluna de dias úteis
    base_calculo['DIAS_UTEIS_CALCULADOS'] = 0
    base_calculo['OBSERVACOES'] = ''
    
    for idx, row in base_calculo.iterrows():
        matricula = row['MATRICULA']
        sindicato = row.get('Sindicato', '')
        
        # Buscar dias úteis do sindicato
        dias_base = 22  # Valor padrão
        for sind_key, dias in dias_uteis_sindicato.items():
            if sind_key in str(sindicato):
                dias_base = dias
                break
        
        dias_calculados = dias_base
        observacoes = []
        
        # Descontar férias
        if pd.notna(row.get('DIAS DE FÉRIAS')):
            dias_ferias = int(row['DIAS DE FÉRIAS'])
            dias_calculados = max(0, dias_calculados - dias_ferias)
            observacoes.append(f"Férias: -{dias_ferias} dias")
        
        # Verificar desligamento
        if pd.notna(row.get('DATA DEMISSÃO')):
            data_demissao = pd.to_datetime(row['DATA DEMISSÃO'])
            comunicado = row.get('COMUNICADO DE DESLIGAMENTO', '')
            
            if data_demissao.day <= 15 and comunicado == 'OK':
                dias_calculados = 0
                observacoes.append("Desligado até dia 15 - sem benefício")
            elif data_demissao.day > 15:
                # Cálculo proporcional
                dias_proporcionais = int(dias_base * (data_demissao.day / 30))
                dias_calculados = min(dias_calculados, dias_proporcionais)
                observacoes.append(f"Desligado dia {data_demissao.day} - proporcional")
        
        base_calculo.at[idx, 'DIAS_UTEIS_CALCULADOS'] = dias_calculados
        base_calculo.at[idx, 'OBSERVACOES'] = '; '.join(observacoes)
    
    print(f"dias úteis calculados para {len(base_calculo)} colaboradores")
    
    return base_calculo

# Calcular dias úteis
base_calculo = calcular_dias_uteis_colaborador(base_principal, dias_uteis_sindicato)

dias úteis calculados para 1795 colaboradores


## 10. Cálculo de Valores de VR

In [11]:
def calcular_valores_vr(base_calculo, valores_sindicato):
    """Calcula os valores de VR para cada colaborador"""
    print("💰 Calculando Valores de VR\n")
    
    base_final = base_calculo.copy()
    
    # Inicializar colunas de valores
    base_final['VALOR_DIARIO_VR'] = 0.0
    base_final['VALOR_TOTAL_VR'] = 0.0
    base_final['CUSTO_EMPRESA'] = 0.0
    base_final['DESCONTO_COLABORADOR'] = 0.0
    
    # Mapeamento de estados por sindicato
    mapa_estado = {
        'SP': 37.5,
        'RJ': 35.0,
        'RS': 35.0,
        'PR': 35.0
    }
    
    for idx, row in base_final.iterrows():
        sindicato = str(row.get('Sindicato', ''))
        dias_uteis = row['DIAS_UTEIS_CALCULADOS']
        
        # Determinar valor diário baseado no sindicato
        valor_diario = 35.0  # Valor padrão
        
        if 'SP' in sindicato or 'SÃO PAULO' in sindicato.upper():
            valor_diario = 37.5
        elif any(estado in sindicato for estado in ['RJ', 'RIO DE JANEIRO']):
            valor_diario = 35.0
        elif any(estado in sindicato for estado in ['RS', 'RIO GRANDE DO SUL']):
            valor_diario = 35.0
        elif any(estado in sindicato for estado in ['PR', 'PARANÁ', 'CURITIBA']):
            valor_diario = 35.0
        
        # Calcular valores
        valor_total = valor_diario * dias_uteis
        custo_empresa = valor_total * 0.8  # 80%
        desconto_colaborador = valor_total * 0.2  # 20%
        
        base_final.at[idx, 'VALOR_DIARIO_VR'] = valor_diario
        base_final.at[idx, 'VALOR_TOTAL_VR'] = valor_total
        base_final.at[idx, 'CUSTO_EMPRESA'] = custo_empresa
        base_final.at[idx, 'DESCONTO_COLABORADOR'] = desconto_colaborador
    
    # Estatísticas
    total_colaboradores = len(base_final[base_final['VALOR_TOTAL_VR'] > 0])
    valor_total_empresa = base_final['CUSTO_EMPRESA'].sum()
    valor_total_desconto = base_final['DESCONTO_COLABORADOR'].sum()
    
    print(f"✅ Valores calculados:")
    print(f"Colaboradores com benefício: {total_colaboradores}")
    print(f"Custo total empresa: R$ {valor_total_empresa:,.2f}")
    print(f"Desconto total colaboradores: R$ {valor_total_desconto:,.2f}")
    print(f"Valor total VR: R$ {valor_total_empresa + valor_total_desconto:,.2f}")
    
    return base_final

# Calcular valores de VR
base_final = calcular_valores_vr(base_calculo, valores_sindicato)

💰 Calculando Valores de VR



✅ Valores calculados:
Colaboradores com benefício: 1773
Custo total empresa: R$ 1,058,948.00
Desconto total colaboradores: R$ 264,737.00
Valor total VR: R$ 1,323,685.00


## 11. Geração da Planilha Final

In [12]:
def gerar_planilha_final(base_final):
    """Gera a planilha final no formato esperado"""
    print("📄 Gerando Planilha Final\n")
    
    # Filtrar apenas colaboradores com benefício
    df_final = base_final[base_final['VALOR_TOTAL_VR'] > 0].copy()
    
    # Selecionar e renomear colunas conforme modelo
    colunas_finais = {
        'MATRICULA': 'Matricula',
        'VALOR_DIARIO_VR': 'VALOR DIÁRIO VR',
        'DIAS_UTEIS_CALCULADOS': 'Dias',
        'VALOR_TOTAL_VR': 'TOTAL',
        'CUSTO_EMPRESA': 'Custo empresa',
        'DESCONTO_COLABORADOR': 'Desconto profissional',
        'OBSERVACOES': 'OBS GERAL'
    }
    
    planilha_final = df_final[list(colunas_finais.keys())].rename(columns=colunas_finais)
    
    # Adicionar colunas adicionais se necessário
    planilha_final['Competência'] = '2025-05-01'
    
    # Reordenar colunas
    ordem_colunas = [
        'Matricula',
        'Competência',
        'Dias',
        'VALOR DIÁRIO VR',
        'TOTAL',
        'Custo empresa',
        'Desconto profissional',
        'OBS GERAL'
    ]
    
    planilha_final = planilha_final.reindex(columns=ordem_colunas)
    
    # Salvar arquivo
    nome_arquivo = f"VR_FINAL_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    
    with pd.ExcelWriter(nome_arquivo, engine='openpyxl') as writer:
        # Aba principal
        planilha_final.to_excel(writer, sheet_name='VR_FINAL', index=False)
        
        # Aba de resumo
        resumo = pd.DataFrame({
            'Métrica': [
                'Total de Colaboradores',
                'Custo Total Empresa (R$)',
                'Desconto Total Colaboradores (R$)',
                'Valor Total VR (R$)'
            ],
            'Valor': [
                len(planilha_final),
                planilha_final['Custo empresa'].sum(),
                planilha_final['Desconto profissional'].sum(),
                planilha_final['TOTAL'].sum()
            ]
        })
        resumo.to_excel(writer, sheet_name='RESUMO', index=False)
        
        # Aba de exclusões
        if motivos_exclusao:
            df_exclusoes = pd.DataFrame([
                {'MATRICULA': mat, 'MOTIVO': motivo} 
                for mat, motivo in motivos_exclusao.items()
            ])
            df_exclusoes.to_excel(writer, sheet_name='EXCLUSOES', index=False)
    
    print(f"✅ Planilha final salva: {nome_arquivo}")
    print(f"📊 Registros na planilha final: {len(planilha_final)}")
    #TODO deixar no padrão da tabela vrmensal
    return planilha_final, nome_arquivo

# Gerar planilha final
planilha_final, nome_arquivo = gerar_planilha_final(base_final)

📄 Gerando Planilha Final

✅ Planilha final salva: VR_FINAL_20250827_211311.xlsx
📊 Registros na planilha final: 1773


## 12. Validações e Relatório Final